In [2]:
import pandas as pd
import numpy as np
import import_ipynb
from my_libs.critical_temp_after_spryer import calc_critical_T_after_spryer

def new_spr(df):
    for x in df['диаметр']:
        if x < 114:
            df['длина трубы'] = df['Скорость прохождения трубы через спрейер, м/с'][0]*(df['шаг балок закалочная печь, сек'][0] - 8)
            df['min скорость спрейера'] = df['длина трубы'][0]/(df['шаг балок закалочная печь, сек']-8)
        else:
            df['длина трубы'] = df['Скорость прохождения трубы через спрейер, м/с'][0]*(2*df['шаг балок закалочная печь, сек'][0] - 8)
            df['min скорость спрейера'] = df['длина трубы'][0]/(2*df['шаг балок закалочная печь, сек']-8)
    df['Скорость прохождения трубы через спрейер, м/с'][1:] = df['min скорость спрейера'][1:]
    return df

def del_bath(df):
    '''Очистка данных с закалкой в ванне'''
#     df = df[pd.isnull(df['t˚ C трубы после ванны'])]
#     df = df[pd.isnull(df['время выдержки в закалочной ванне, сек.'])]
    del df['t˚ C трубы после ванны']
    del df['время выдержки в закалочной ванне, сек.']
    return df

def clean_data(file_name, ls_need_col):
    '''Доочистка prepared файлов'''
    df = pd.read_excel(file_name)
    df = calc_all_features(df)
    try:
        df = df[ls_need_col+['Предел текучести', 'Врем. сопротивление', '№ партии']]
    except KeyError:
        df = df[ls_need_col]
        print('Warning!, Предел текучести, Врем. сопротивление, № партии Not in index')
    try:
        df = del_bath(df)
    except:
        pass
    print(df.shape)
    df = df.dropna()
    print(df.shape)
    return df

def clean_data_sheets(df, ls_need_col):
    '''Доочистка prepared файлов'''
    df = calc_all_features(df)
    df = df[ls_need_col+['Предел текучести', 'Врем. сопротивление', '№ партии']]
    try:
        df = del_bath(df)
    except:
        pass
    print(df.shape)
    df = df.dropna()
    print(df.shape)
    return df

def ideal_critical_d(df):
    """Считает идеальный критический диаметр, возвращает датафрейм с добавленным признаком"""
    df['k1'] = df['C'].apply(lambda x:  0.54*x if x <= 0.39 else 0.171+0.001*x+0.265*x*x)
    df['k2'] = df['Mn'].apply(lambda x: 3.3333*x+1 if x <= 1.2 else 5.1*x-1.12)
    df['ICD'] = df['k1']*df['k2']*(1+0.7*df['Si'])*(1+0.363*df['Ni'])*(1+2.16*df['Cr'])*(
        1+3.0*df['Mo'])*(1+0.365*df['Cu'])*(1+1.73*df['V'])
    del df['k1']
    del df['k2']
    return df

def calc_C_coef(df):
    """Считает углеродный коэффициент, возвращает датафрейм с добавленным признаком"""
    df['C-coef'] = (df['C'] + df['Mn']/6.0 + (df['Cr'] + df['Mo'] + df['V'])/5.0 +
                        (df['Ni'] + df['Cu'])/15.0)
    return df

def calc_quenching_param(df):
    """Считает параметр закалки, возвращает датафрейм с добавленным признаком"""
    df['Параметр закалка'] = 1/(1/(df[['3 зона по ВТР закалка']].mean(axis=1) 
                            + 273) - (2.303*1.986*np.log10(61*df['шаг балок закалочная печь, сек']/3600)/110000))-273
    return df

def calc_tempering_param(df):
    """Считает параметр отпуска, возвращает датафрейм с добавленным признаком"""
    df['Параметр отпуск'] = (df[['3 зона ВТР и уставка отпуск','4 зона ВТР и уставка отпуск','5 зона ВТР и уставка отпуск']].mean(axis=1)
                        + 273)*(20+np.log(94*df['шаг балок отпускная печь, сек']/3600))*1e-3
    return df

def calc_tempering_param_new(df):
    """Считает параметр отпуска усовершенствованный (с зависимостью от химии), возвращает датафрейм с добавленным признаком"""
    df['Параметр отпуск новый'] = (np.log(94*df['шаг балок отпускная печь, сек']/3600)-(
        (114.7*(df['Mo']+df['Mn']/5+df['Cr']/10)+46.6)/4.6/(
        df[['2 зона ВТР и уставка отпуск', '3 зона ВТР и уставка отпуск','4 зона ВТР и уставка отпуск','5 зона ВТР и уставка отпуск']].mean(axis=1)
                        + 273))+50)
    return df

def calc_tempering_param_new_2(df):
    """Считает параметр отпуска, возвращает датафрейм с добавленным признаком"""
    df['Параметр отпуск новый 2'] = (df[['3 зона ВТР и уставка отпуск','4 зона ВТР и уставка отпуск','5 зона ВТР и уставка отпуск']].mean(axis=1)
                        + 273)*((
        17.369-6.661*df['C']-1.604*df['Mn']-3.412*df['Si']-0.248*df['Ni']-1.112*df['Cr']-4.355*df['Mo'])+
        np.log(150*df['шаг балок отпускная печь, сек']/3600))*1e-3
    return df

def calc_ratio(df):
    '''считаем отношение предела текучести к пределу прочности'''
    df['Отношение'] = df['Предел текучести']/df['Врем. сопротивление']
    return df

"""Меняли длительность отпуска было 94"""


def calc_tempering_param_new_V(df):
    """Считает параметр отпуска, возвращает датафрейм с добавленным признаком"""
    df['Параметр отпуск новый V'] = (df[['3 зона ВТР и уставка отпуск','4 зона ВТР и уставка отпуск','5 зона ВТР и уставка отпуск']].mean(axis=1)
                        + 273)*((
        17.369-6.661*df['C']-1.604*df['Mn']-3.412*df['Si']-0.248*df['Ni']-1.112*df['Cr']-4.355*df['Mo']-2.6*df['V'])+
        np.log(94*df['шаг балок отпускная печь, сек']/3600))*1e-3
    return df

def calc_seed_size(df):
    chem_dict = {
        'X' : ['Ni', 'Cr', 'Mo', 'Cu', 'Al', 'V', 'Ti', 'Nb', 'N'],
        'p' : [-1.729, 0, 0.223, -7.449, 18.996, 1.35, -22.399, 148.515, -410.616],
        'q' : [-16185, -1132, 2540, -41949, 222088, 39721, -208274, 1721974, -4321929]
    }


    #   случайно поставила
    T = df[['2 зона по ВТР закалка', '3 зона по ВТР закалка']].mean(axis=1) 
    df['T'] = T
    #   случайно поставила
    t = 65

    df['sum_piXi'] = 0
    df['sum_qiXi'] = 0
    for i in range(0,len(chem_dict['X'])):
        df['sum_piXi']+=df[chem_dict['X'][i]]*chem_dict['p'][i]
        df['sum_qiXi']+=df[chem_dict['X'][i]]*chem_dict['q'][i]

    df['Величина зерна'] = np.exp(10.313+0.93*df['C']*(1-0.469*df['Cr'])+df['sum_piXi'])*(
    np.exp(-(77978+3116*df['C']+df['sum_qiXi'])/8.314/(T+273))*((t*df['шаг балок закалочная печь, сек'])**0.107))
    
    return df

def calc_all_features(df):
    """Считает и добавляет все необходимые признаки в датафрейм"""
    df = ideal_critical_d(df)
    df = calc_C_coef(df)
    df = calc_quenching_param(df)
    df = calc_tempering_param(df)
    df = calc_tempering_param_new(df)
    df = calc_tempering_param_new_2(df)
    df = calc_tempering_param_new_V(df)
#     df = calc_ratio(df)
    df = calc_seed_size(df)
    df = calc_critical_T_after_spryer(df)
    df = new_spr(df)
    return df

def make_prepared(df, exp_df, chem):
    """Подготавливает датафрейм"""
#     обединяем испытания с режимами и химией
    df_merge = pd.merge(exp_df, df, how ='left', on = ['№ плавки','№ партии'])
    print('обединяем испытания с режимами и химией', df_merge.shape)
#     заполняем пустую химию средней (или нулями)
    df_merge = df_merge.fillna(value = chem)
    print('заполняем пустую химию средней (или нулями)', df_merge.shape)
#     удаляем строки без режимов
    df_merge = df_merge[~pd.isnull(df_merge['C'])]
    print('удаляем строки без химии', df_merge.shape)
#     добавляем рассчитаные признаки
    df_merge = calc_all_features(df_merge)
    print('cчитаем все признаки', df_merge.shape)
    df_merge = df_merge[df_merge['Cr'] < 10]
    print('Сr>10 del ', df_merge.shape)
    df_merge.reset_index(drop=True, inplace=True)
    print(df_merge.shape)
    return df_merge

ls_need_col = [
#  '№ плавки',
#  '№ партии',
#  'НД (гост, ту)',
#  'марка стали',
 '1 зона по ВТР закалка',
 '2 зона по ВТР закалка',
 '3 зона по ВТР закалка',
 'Скорость прохождения трубы через спрейер, м/с',
 't˚ C трубы после спреера',
 'время выдержки в закалочной ванне, сек.',
 't˚ C трубы после ванны',
 '1 зона ВТР и уставка отпуск',
 '2 зона ВТР и уставка отпуск',
 '3 зона ВТР и уставка отпуск',
 '4 зона ВТР и уставка отпуск',
 '5 зона ВТР и уставка отпуск',
 'шаг балок закалочная печь, сек',
 'шаг балок отпускная печь, сек',
 'диаметр',
 'толщина стенки',
#  'Поставщик',
 'C',
 'Mn',
 'Si',
 'P',
 'S',
 'Cr',
 'Ni',
 'Cu',
 'Al',
 'V',
 'Ti',
 'Nb',
 'Mo',
 'N',
#  'B',
#  'Тип предела текучести (1186)',
 'Параметр закалка',
#  'Параметр отпуск',
 'Параметр отпуск новый',
#  'Параметр отпуск новый 2',
 'Параметр отпуск новый V',
 'C-coef',
 'ICD',
 'Величина зерна'
]


ModuleNotFoundError: No module named 'my_libs'